<a href="https://colab.research.google.com/github/KoreaUnivseokjinmoon/seokjinmoonNLU/blob/main/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC(NSMC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. BERT 모델을 다운로드
BERT-Base, Multilingual Cased: 104 languages, 12-layer, 768-hidden, 12-heads, 110M parameters

In [12]:
# wget을 활용해서 bert 모델 다운로드 가능
import os
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import zipfile
import shutil
         
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')
 
bert_zip.close()

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("bert/multi_cased_L-12_H-768_A-12", "bert")

--2020-12-24 12:45:53--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 172.217.7.208, 172.217.9.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip.3’

multi_cased_L-12_H- 100%[===================>] 632.19M   198MB/s    in 3.2s    

2020-12-24 12:45:56 (198 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip.3’ saved [662903077/662903077]



2. 구글 Drive와 Google Colab 연동

In [13]:
import os
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [14]:
#os.listdir('gdrive/My Drive/Korea_Univ_NLP')

3. BERT 모델이 있는 Path를 변수로 지정

In [15]:
path = "gdrive/My Drive/Korea_Univ_NLP"
path

'gdrive/My Drive/Korea_Univ_NLP'

4. 프로젝트 시 필요한 모듈 Import

In [16]:
%tensorflow_version 2.x
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

5. BERT 모델 활용을 위한 keras-bert & 
Adam optimizer의 수정판인 keras-radam 모듈을 임포트

In [17]:
!pip install keras-bert

6. keras-bert 라이브러리에서 BERT 모형 활용에 필요한 모듈들을 임포트

In [18]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

#from keras_radam import RAdam

In [19]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [20]:
os.listdir('nsmc')

['raw',
 'ratings_test.txt',
 'ratings.txt',
 'README.md',
 'code',
 '.git',
 'synopses.json',
 'ratings_train.txt']

In [21]:
!git clone https://github.com/KoreaUnivseokjinmoon/seokjinmoonNLU.git

Cloning into 'seokjinmoonNLU'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 25 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [22]:
os.listdir('seokjinmoonNLU')

['friends_en', 'README.md', 'testSet', '.git']

BERT 모델을 훈련시킬 데이터를 Train Data와 Test Data로 나눈다.

In [23]:
train = pd.read_table("nsmc/"+"ratings_train.txt")
test = pd.read_table("nsmc/"+"ratings_test.txt")

temp = pd.read_csv("seokjinmoonNLU/testSet/"+"ko_data.csv", encoding='CP949')

#train = pd.concat([train, test])


temp.columns = ['id', 'document']
test = temp

train.shape

(150000, 3)

In [24]:
train[149998:150010]

,id,document,label
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [25]:
test[50:70]

,id,document
50,50,너무 야하다
51,51,더 이상 숨어서는 안 된다.그냥 지나갈 뿐...
52,52,로맨스이긴 하지만 내용의 흐림이 어쩐지 어색한 것
53,53,"글쎄요, 너무 평범해서 시간낭비하고 싶어요."
54,54,허버 굳이 미야이즈미 죽이지마 -- 죽일 때가 기억에 남지 마
55,55,10점이 아깝지 않다
56,56,인권이라는 주제를 잘 표현한 영화
57,57,천재적인 아이들이 마음에 끌렸다고 생각합니다.~^-^!!
58,58,"이거는 거의 플루시맨을 보는 것 같은... 무슨 영화가 뭐야, 그래픽도."
59,59,가족호러에 에디머피가 나오면 재미도 없고 호러도 아니라는 결론이 나온다


BERT 훈련을 위한 사전 설정
SEQ_LEN : 문장의 최대 길이
BATCH_SIZE : 메모리 초과 같은 문제를 방지하기 위해 작은 수인 16으로 정했습니다.
EPOCHS : 총 훈련 에포크 수
LR : 학습률, Learning rate
pretrained_path는 bert 사전학습 모형이 있는 폴더를 의미합니다.
그리고 우리가 분석할 문장이 들어있는 칼럼의 제목인 document와 긍정인지 부정인지 알려주는 칼럼을 label로 정해줍니다

In [26]:
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5
 
pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
 
DATA_COLUMN = "document"
LABEL_COLUMN = "label"

In [27]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [28]:
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [29]:
tokenizer = inherit_Tokenizer(token_dict)

In [30]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        targets.append(data_df[LABEL_COLUMN][i])
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [31]:
train_x, train_y = load_data(train)
#test_x, test_y = load_data(test)

100%|██████████| 150000/150000 [00:19<00:00, 7698.83it/s]


In [32]:
def sentence_convert_data(data): 
    global tokenizer
    indices = []
    for i in tqdm(range(len(data))):
        print(tokenizer.tokenize(data[i]))
        ids, segments = tokenizer.encode(data[i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def sentence_load_data(sentences):#sentence는 List로 받는다
           
    data_x = sentence_convert_data(sentences)

    return data_x

In [33]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 128)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 128, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 128, 768)     1536        Input-Segment[0][0]              
______________________________________________________________________________________________

In [35]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output
 
 
  outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)
 
 
 
  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=tf.optimizers.Adam(learning_rate=0.00001, beta_1=0.9),
      loss='binary_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

In [ ]:
bert_model = get_bert_finetuning_model(model)
 
history = bert_model.fit(train_x, train_y, epochs=3, batch_size=32, verbose = 1, validation_split=0.25, shuffle=True)

Epoch 1/3
  84/3516 [..............................] - ETA: 1:27:37 - loss: 0.6751 - accuracy: 0.5701

In [ ]:
 #bert_model.save_weights(path+"/bertv2.h5")

In [ ]:
bert_model = get_bert_finetuning_model(model)
#bert_model.load_weights(path+"/bertv2.h5")

In [ ]:
def predict_convert_data(data_df):
    global tokenizer
    indices = []
    for i in tqdm(range(len(data_df))):
        ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
        indices.append(ids)
        
    items = indices
    
    
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def predict_load_data(x): #Pandas Dataframe을 인풋으로 받는다
    data_df = x
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)


    data_x = predict_convert_data(data_df)

    return data_x

In [ ]:
test_set = predict_load_data(test)

In [ ]:
preds = ( bert_model.predict(test_set) > 0.5 ).astype("int32")
#preds = np.argmax(bert_model.predicttest_set)
#preds = bert_model.predict_classes(test_set, verbose = 0)

In [ ]:
preds

In [ ]:
testCSV = test.copy()
del testCSV['document']
result = pd.DataFrame(preds)
result

finCSV = pd.concat([testCSV, result], axis=1)
finCSV.columns = ['id', 'Predicted']
finCSV.to_csv("sample.csv", mode='w', index = False)



# 새 섹션